# Adopt Timeliness

## Contents
#### Setup
1. [Import packages and options](#import_packages)
2. [Define key variables](#define_key_variables)

3. [Adopt_applications_data_sorted](#Adopt_applications_data_sorted) - Orders adopt_application_5 by case_number and application date
4. [Adopt_applications_temp](#Adopt_applications_temp) - takes the adopt_applications_data sorted table and filters it so it only has the first application record per case number
5. [adopt_orders_data_sorted](#adopt_orders_data_sorted) - Sorts the disposals table by case_number and receipt date, removing contact orders, placement revoke or vary orders and other type orders
6. [adopt_orders_temp](#adopt_orders_temp) - takes the adopt_orders_data_sorted table and filters it so it only has the first disposal record per case number
7. [adopt_apps_and_orders_match](#adopt_apps_and_orders_match) - calculates the wait_weeks from the disposal date and the application date


8. [Adopt_case_data_temp](#Adopt_case_data_temp) - filters by year>2010 and by first application date
9. [Applicant_Info](#Applicant_Info) - joins the roles, parties and address tables for applicants to get information on the applicants
10. [adopt_respondent_info](#adopt_respondent_info) - joins the roles, parties and address tables for respondents to get information on the respondents
11. [applicants_3](#applicants_3) takes the adopt_applicant_info table and reformats it, decoding gender and representative role values into strings

12. [Adopt_respondents_temp](#Adopt_respondents_temp) - takes the adopt_respondent_info table and reformats it, decoding gender and representative role values into strings
13. [Adopt_app_rep](#Adopt_app_rep) - joins the case data with the applicant representation data
14. [Adopt_resp_rep](#Adopt_resp_rep) - joins the case data with the respondent representation data
15. [Adopt_Hearing_Events](#Adopt_Hearing_Events) - takes the adoption hearing events from the hearings table and joins it with the events table data 
16. [Adopt_Hearings_Cases](#Adopt_Hearings_Cases) - takes the hearing events, filters by adoption event codes and adds a flag whether the hearing is the first in the case
17. [Hearing_Adopt_Applicants](#Hearing_Adopt_Applicants) - joins the cases from applicant representation table to the first hearing for the case in the adopt hearings cases table

18. [Hearing_Adopt_Respondents](#Hearing_Adopt_Respondents) - joins the cases from respondents representation table to the first hearing for the case in the adopt hearings cases table 
19. [Adopt_App](#Adopt_App) - Groups the hearing_adopt_applicants table and produces a count per group
20. [Adopt_Resp](#Adopt_Resp) - Groups the hearing_adopt_respondents table and produces a count per group
21. [adopt_case](#adopt_case) - groups and formats adopt_case_data_v3 table and gives a count for each group
22. [Adopt_Case_Hearings](#Adopt_Case_Hearings) - creates a count of all the cases with a hearing per quarter
23. [Adoption](#Adoption) - Joins the applicant/respondent representation count tables, case count table, and case hearing count tables 
24. [Applicant_representation](#Applicant_representation) - creates a table showing whether all, some, or none of the applicants for a case have representation
25. [Respondent_representation](#Respondent_representation) - creates a table showing whether all, some, or none of the respondents for a case have representation
26. [Adopt_Disposals_Final](#Adopt_Disposals_Final) - joins the adopt_apps_and_orders_match table and the representation tables, joining timeliness and legal representation data for a case together
27. [Adopt_Quarterly](#Adopt_Quarterly) - groups the disposals_final table by quarter, providing a total and mean wait_weeks per quarter
28. [Adopt_Annual](#Adopt_Annual) - groups the disposals_final table annually, providing a total and mean wait_weeks annually 
29. [Adopt_timeliness_combined](#Adopt_timeliness_combined) - combines and orders the annual and quarterly timeliness data


## 1. Import packages and set options 
<a name="import_packages"></a>

In [1]:
import pandas as pd  # a module which provides the data structures and functions to store and manipulate tables in dataframes
import pydbtools as pydb  # A module which allows SQL queries to be run on the Analytical Platform from Python, see https://github.com/moj-analytical-services/pydbtools
import boto3  # allows you to directly create, update, and delete AWS resources from Python scripts

# sets parameters to view dataframes for tables easier
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)


## 2. Define key variables to be used throughout the notebook 
<a name="define_key_variables"></a>

In [2]:
#this is the database we will be extracting from
database = "familyman_dev_v2"

#this is the snapshot date of familyman we will be extracting from
snapshot_date = "2021-08-19"

#this is the athena database we will be storing our tables in
fcsq_database = "fcsq"

#this is the s3 bucket we will be saving data to
s3 = boto3.resource("s3")
bucket = s3.Bucket("alpha-family-data")

In [3]:
"""
CREATE EXTERNAL TABLE IF NOT EXISTS `fcsq`.`COURT_MV_FEB21_DFJ` (
  `CODE` int,
  `COURT` string,
  `NAME` string,
  `Court_Type` string,
  `DFJ_Area` string,
  `Region` string,
  `Court_pre_2014` string,
  `Region_Pre2014` string,
  `DFJ_New` string
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe' 
WITH SERDEPROPERTIES (
  'serialization.format' = ',',
  'field.delim' = ','
) LOCATION 's3://alpha-family-data/fcsq_processing/lookups/COURT_MV_FEB21_DFJ/'
TBLPROPERTIES ('has_encrypted_data'='false');
"""

"\nCREATE EXTERNAL TABLE IF NOT EXISTS `fcsq`.`COURT_MV_FEB21_DFJ` (\n  `CODE` int,\n  `COURT` string,\n  `NAME` string,\n  `Court_Type` string,\n  `DFJ_Area` string,\n  `Region` string,\n  `Court_pre_2014` string,\n  `Region_Pre2014` string,\n  `DFJ_New` string\n)\nROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe' \nWITH SERDEPROPERTIES (\n  'serialization.format' = ',',\n  'field.delim' = ','\n) LOCATION 's3://alpha-family-data/fcsq_processing/lookups/COURT_MV_FEB21_DFJ/'\nTBLPROPERTIES ('has_encrypted_data'='false');\n"

## 3. Adopt_applications_data_sorted table - Orders adopt_application_5 by case_number and application date
<a name="Adopt_applications_data_sorted"></a>

### Drop the adopt_applications_data_sorted table if it already exists and remove its data from the S3 bucket

In [4]:
drop_adopt_applications_data_sorted = f"""
DROP TABLE IF EXISTS fcsq.adopt_applications_data_sorted;
"""
pydb.start_query_execution_and_wait(drop_adopt_applications_data_sorted)

# clean up previous adopt_applications_data_sorted files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/adopt_applications_data_sorted/").delete();

### Create the adopt_applications_data_sorted table in Athena

In [5]:
create_adopt_applications_data_sorted = f"""
CREATE TABLE IF NOT EXISTS fcsq.adopt_applications_data_sorted
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/adopt_applications_data_sorted') AS
SELECT * FROM
fcsq.adopt_application_5
ORDER BY 
CASE_NUMBER, APP_DATE
"""

pydb.start_query_execution_and_wait(create_adopt_applications_data_sorted);



#### adopt_applications_data_sorted validation

In [6]:
adopt_applications_data_sorted_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_applications_data_sorted")
adopt_applications_data_sorted_count


,count
0,175169


## 4. Create adopt_applications temporary tables - takes the adopt_applications_data sorted table and filters it so it only has the first application record per case number
<a name="Adopt_applications_temp"></a>

In [7]:
create_adopt_applications_1 = f"""
SELECT *, row_number() over (order by CASE_NUMBER, APP_DATE) as SEQ_NUM
FROM fcsq.adopt_applications_data_sorted
"""
pydb.create_temp_table(create_adopt_applications_1,'adopt_applications_1')

create_adopt_applications_2 = f"""
SELECT DISTINCT case_number, app_type, min(seq_num) as min_of_seq_num
FROM __temp__.adopt_applications_1 GROUP BY case_number, app_type
"""
pydb.create_temp_table(create_adopt_applications_2,'adopt_applications_2')

create_adopt_applications_3 = f"""
SELECT
    t1.case_number,
    t2.App_date,
    t2.year,
    t2.quarter,
    t2.court,
    t2.app_type,
    t2.Case_app_type, 
    t2.Adoption, 
    t2.Contested, 
    t2.Standard, 
    t2.Convention, 
    t2.Foreign, 
    t2.Placement, 
    t2.Placement_revoke_or_vary, 
    t2.Contact_s26, 
    t2.Contact_s26_revoke_or_vary, 
    t2.Change_surname, 
    t2.Remove_child_from_UK, 
    t2.Other_order_type, 
    t2.Adoption_Cases, 
    t2.Non_Adoption_Cases
FROM __temp__.ADOPT_APPLICATIONS_2 t1 LEFT JOIN
__temp__.ADOPT_APPLICATIONS_1 t2 ON (t1.MIN_of_seq_num = t2.Seq_Num)
"""
pydb.create_temp_table(create_adopt_applications_3,'adopt_applications_3')

## 5. adopt_orders_data_sorted table - Sorts the disposals table by case_number and receipt date, removing contact orders, placement revoke or vary orders and other type orders
<a name="adopt_orders_data_sorted"></a>

### Drop the adopt_orders_data_sorted table if it already exists and remove its data from the S3 bucket

In [8]:
drop_adopt_orders_data_sorted = f"""
DROP TABLE IF EXISTS fcsq.adopt_orders_data_sorted;
"""
pydb.start_query_execution_and_wait(drop_adopt_orders_data_sorted)

# clean up previous adopt_orders_data_sorted files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/adopt_orders_data_sorted/").delete();

### Create the adopt_orders_data_sorted table in Athena

In [9]:
create_adopt_orders_data_sorted = f"""
CREATE TABLE IF NOT EXISTS fcsq.adopt_orders_data_sorted
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/adopt_orders_data_sorted') AS
SELECT * 
FROM fcsq.adopt_disposals5 t1
    WHERE t1.Type != 'Contact_s26' AND t1.Type != 'Contact_s26_revoke_or_vary' AND t1.Type != 
        'Placement_revoke_or_vary' AND t1.Type != 'Other_order_type'
    ORDER BY t1.CASE_NUMBER, t1.Receipt_date;
"""
pydb.start_query_execution_and_wait(create_adopt_orders_data_sorted);

#### adopt_orders_data_sorted validation

In [10]:
adopt_orders_data_sorted_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_orders_data_sorted")
adopt_orders_data_sorted_count

,count
0,150507


In [169]:
pydb.read_sql_query("SELECT * FROM fcsq.adopt_orders_data_sorted")
pydb.read_sql_query("SELECT * FROM __temp__.adopt_orders_1 where case_number='LV20Z04103'")

,type
0,Remove_child_from_UK
1,Change_surname
2,
3,Convention
4,?Foreign?
5,Placement
6,Standard


## 6. Create adopt_orders temporary tables - takes the adopt_orders_data_sorted table and filters it so it only has the first disposal record per case number
<a name="adopt_orders_temp"></a>

In [11]:
create_adopt_orders_0 = f"""
SELECT t1.CASE_NUMBER, 
          t1.Court, 
          t1.Year, 
          t1.Quarter, 
          t2.App_type, 
          t2.App_date, 
          t1.Receipt_date AS Disp_Date, 
          t1.EVENT_MODEL, 
          t1.FIELD_MODEL, 
          t1.Order_type, 
          t1.Country_of_birth, 
          t1.Number_applicants, 
          t1.Adopter_type, 
          t1.Adopter, 
          t1.Child_sex, /*Changed this from child_sex2 as the process has changed, check to see if an error comes up*/ 
          t1.Age_band, 
          t1.Child_age, 
          t1.Adoption, 
          t1.Type, 
          t1.Adopter_2, 
          DAY(t1.Receipt_date -t2.App_date) as DATE_DIFF
      FROM fcsq.ADOPT_ORDERS_DATA_SORTED t1
           INNER JOIN __temp__.ADOPT_APPLICATIONS_3 t2 ON (t1.CASE_NUMBER = t2.CASE_NUMBER)
                 WHERE (DAY(t1.Receipt_date -t2.App_date)) >= 0;
"""
pydb.create_temp_table(create_adopt_orders_0,'adopt_orders_0')


create_adopt_orders_1 = f"""
   SELECT t1.CASE_NUMBER, 
          t1.Court, 
          t1.Year, 
          t1.Quarter, 
          t1.App_type, 
          t1.EVENT_MODEL AS Disp_Type, 
          t1.App_date, 
          t1.Disp_Date, 
          t1.FIELD_MODEL, 
          t1.Order_type, 
          t1.Number_applicants, 
          t1.Child_sex, /*Changed this from child_sex2 as the process has changed, check to see if an error comes up*/ 
          t1.Adoption, 
          t1.Type, 
          row_number() over (order by CASE_NUMBER,App_type,App_date,disp_date) as Seq_no
      FROM __temp__.ADOPT_ORDERS_0 t1
      ORDER BY t1.CASE_NUMBER,
               t1.App_type,
               t1.App_date;
"""

pydb.create_temp_table(create_adopt_orders_1,'adopt_orders_1')

create_adopt_orders_2 = f"""
SELECT DISTINCT CASE_NUMBER, 
          App_type, 
          App_date, 
        (MIN(Seq_No)) AS MIN_of_Seq_No
      FROM __temp__.ADOPT_ORDERS_1 t1
      GROUP BY CASE_NUMBER,
               App_type,
               App_date;
"""

pydb.create_temp_table(create_adopt_orders_2,'adopt_orders_2')

create_adopt_orders_3 = f"""
   SELECT DISTINCT t1.CASE_NUMBER, 
          t1.App_type, 
          t2.Court, 
          t1.App_date, 
          t2.Disp_Date, 
          t2.Disp_Type, 
          t2.Year, 
          t2.Quarter, 
          t2.Order_type, 
          t2.Adoption, 
          t2.Type
      FROM __temp__.ADOPT_ORDERS_2 t1
           LEFT JOIN __temp__.ADOPT_ORDERS_1 t2 ON (t1.MIN_of_Seq_No = t2.Seq_No);
"""
pydb.create_temp_table(create_adopt_orders_3,'adopt_orders_3')


,case_number,court,year,quarter,app_type,disp_type,app_date,disp_date,field_model,order_type,number_applicants,child_sex,adoption,type,seq_no
0,LV20Z04103,251,2021,1,AO,A76,2020-11-25,2021-02-18,<NA>,<NA>,2,2,Adoption,Standard,66778


In [132]:
pydb.read_sql_query("SELECT * FROM __temp__.adopt_orders_1 where case_number='MB20Z01235'")

,case_number,court,year,quarter,app_type,disp_type,app_date,disp_date,field_model,order_type,number_applicants,child_sex,adoption,type,seq_no
0,MB20Z01235,270,2020,4,AO,A73,2020-07-25,2020-10-06,A73_1,OFC,2,1,Non-adoption,,78622
1,MB20Z01235,270,2020,4,AO,A76,2020-07-25,2020-10-06,<NA>,<NA>,2,1,Adoption,Standard,78623


In [123]:
pydb.read_sql_query("SELECT * FROM __temp__.adopt_orders_1 where case_number='MB20Z01235'")

,case_number,court,year,quarter,app_type,disp_type,app_date,disp_date,field_model,order_type,number_applicants,child_sex,adoption,type,seq_no
0,MB20Z01235,270,2020,4,AO,A73,2020-07-25,2020-10-06,A73_1,OFC,2,1,Non-adoption,,78622
1,MB20Z01235,270,2020,4,AO,A76,2020-07-25,2020-10-06,<NA>,<NA>,2,1,Adoption,Standard,78623


In [125]:
pydb.read_sql_query("SELECT * FROM __temp__.adopt_orders_1 where case_number='WX20Z00011'")

,case_number,court,year,quarter,app_type,disp_type,app_date,disp_date,field_model,order_type,number_applicants,child_sex,adoption,type,seq_no
0,WX20Z00011,384,2020,4,AO,A76,2020-03-09,2020-11-06,<NA>,<NA>,1,1,Adoption,Standard,144445


In [ ]:
test = f"""
   SELECT DISTINCT t1.CASE_NUMBER,
          t1.MIN_of_Seq_No,
          t1.App_type, 
          t2.Court, 
          t1.App_date, 
          t2.Disp_Date, 
          t2.Disp_Type, 
          t2.Year, 
          t2.Quarter, 
          t2.Order_type, 
          t2.Adoption, 
          t2.Type
      FROM __temp__.ADOPT_ORDERS_2 t1
           LEFT JOIN __temp__.ADOPT_ORDERS_1 t2 ON (t1.MIN_of_Seq_No = t2.Seq_No)
    where t1.case_number='NN07Z03357';
"""
pydb.read_sql_query(test)

,case_number,min_of_seq_no,app_type,court,app_date,disp_date,disp_type,year,quarter,order_type,adoption,type
0,NN07Z03357,93382,AO,282,2007-05-30,2007-12-21,A76,2007,4,<NA>,Adoption,Standard


## 7. adopt_apps_and_orders_match table - calculates the wait_weeks from the disposal date and the application date
<a name="adopt_apps_and_orders_match"></a>

### Drop the adopt_apps_and_orders_match table if it already exists and remove its data from the S3 bucket

In [14]:
drop_adopt_apps_and_orders_match = f"""
DROP TABLE IF EXISTS fcsq.adopt_apps_and_orders_match;
"""
pydb.start_query_execution_and_wait(drop_adopt_apps_and_orders_match)

# clean up previous adopt_apps_and_orders_match files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/ADOPT_APPS_AND_ORDERS_MATCH/").delete();

### Create the adopt_apps_and_orders_match table in Athena

In [15]:
create_adopt_apps_and_orders_match =f"""
CREATE TABLE IF NOT EXISTS fcsq.ADOPT_APPS_AND_ORDERS_MATCH
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/ADOPT_APPS_AND_ORDERS_MATCH') AS
   SELECT t2.CASE_NUMBER, 
          t2.App_type, 
          t2.Court, 
          t2.App_date, 
          t2.Disp_Date, 
          /* Wait_weeks */
            (CAST (DAY(t2.Disp_Date-t2.App_date) as double)/7) AS Wait_weeks, 
          t2.Disp_Type, 
          t2.Year, 
          t2.Quarter, 
          t2.Order_type, 
          t2.Adoption, 
          t2.Type, 
          /* DSP_COURT */
          t2.Court AS DSP_COURT
      FROM __temp__.ADOPT_ORDERS_3 t2;


"""
pydb.start_query_execution_and_wait(create_adopt_apps_and_orders_match);

#### ADOPT_APPS_AND_ORDERS_MATCH validation

In [16]:
adopt_apps_and_orders_match_count = pydb.read_sql_query("select count(*) as count from fcsq.ADOPT_APPS_AND_ORDERS_MATCH")
adopt_apps_and_orders_match_count

,count
0,147875


In [126]:
pydb.read_sql_query("select * from fcsq.adopt_apps_and_orders_match where case_number='WX20Z00011'")

,case_number,app_type,court,app_date,disp_date,wait_weeks,disp_type,year,quarter,order_type,adoption,type,dsp_court
0,WX20Z00011,AO,384,2020-03-09,2020-11-06,34.571429,A76,2020,4,<NA>,Adoption,Standard,384


## 8. Adopt_case_data temporary tables - filters by year>2010 and by first application date
<a name="Adopt_case_data_temp"></a>

In [17]:
create_adopt_case_data_v1 = f"""
SELECT T1.YEAR,
            T1.QUARTER,
            T1.COURT,
            T1.CASE_NUMBER,
            T1.APP_TYPE,
            T1.CASE_APP_TYPE,
            T1.ADOPTION,
            T1.HIGH_COURT,
            T1.CONTESTED,
            T1.NUMBER_APPLICANTS,
            T1.ADOPTER_TYPE,
            date_format(T1.APP_DATE,'%d-%m-%Y') AS APP_DATE2
    FROM fcsq.adopt_apps_6_adoptions_only AS t1
    ORDER BY case_number, app_date2, court;
    
"""

create_adopt_case_data_v2 = f"""
SELECT *,(case when row_number() over (partition by case_number order by 
        APP_DATE2) = 1 then 1 else 0 end) as case_number_id
FROM __temp__.adopt_case_data_v1
"""

create_adopt_case_data_v3 = f"""
SELECT *
FROM __temp__.adopt_case_data_v2
where case_number_id = 1 and year > 2010;
"""
pydb.create_temp_table(create_adopt_case_data_v1,'adopt_case_data_v1')

pydb.create_temp_table(create_adopt_case_data_v2,'adopt_case_data_v2')
pydb.create_temp_table(create_adopt_case_data_v3,'adopt_case_data_v3')





In [128]:
adopt_case_data_v3_check = "SELECT * from __temp__.adopt_case_data_v3 where case_number='WX20Z00011'"
pydb.read_sql_query(adopt_case_data_v3_check)

,year,quarter,court,case_number,app_type,case_app_type,adoption,high_court,contested,number_applicants,adopter_type,app_date2,case_number_id
0,2020,1,384,WX20Z00011,AO,AO,Adoption,N,N,1,single,09-03-2020,1


## 9. Applicant_Info table - joins the roles, parties and address tables for applicants to get information on the applicants
<a name="Applicant_Info"></a>

### Drop the Applicant_Info table if it already exists and remove its data from the S3 bucket

In [19]:
drop_Adopt_Applicant_Info = f"""
DROP TABLE IF EXISTS fcsq.Adopt_Applicant_Info;
"""
pydb.start_query_execution_and_wait(drop_Adopt_Applicant_Info)

# clean up previous Adopt_Applicant_Info files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/Adopt_Applicant_Info/").delete();

### Create the Applicant_Info table in Athena

In [20]:
create_Adopt_Applicant_Info = f"""
CREATE TABLE IF NOT EXISTS fcsq.Adopt_Applicant_Info
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Adopt_Applicant_Info') AS
 SELECT DISTINCT
   {database}.roles.ROLE, 
   {database}.roles.REPRESENTATIVE_ROLE, 
   {database}.roles.ROLE_MODEL, 
   {database}.roles.PARTY, 
   {database}.roles.CASE_NUMBER, 
   {database}.parties.PERSON_GIVEN_FIRST_NAME, 
   {database}.parties.PERSON_FAMILY_NAME, 
   {database}.parties.COMPANY, 
   {database}.addresses.POSTCODE, 
   {database}.parties.GENDER, 
   {database}.roles.DELETE_FLAG
FROM 
  ({database}.roles INNER JOIN {database}.parties ON {database}.roles.PARTY = {database}.parties.PARTY) 
  LEFT JOIN {database}.addresses ON {database}.roles.ADDRESS = {database}.addresses.ADDRESS
WHERE 
    ((({database}.roles.ROLE_MODEL)= 'APLZ') AND (({database}.roles.DELETE_FLAG)= 'N')) 
    OR ((({database}.roles.ROLE_MODEL)= 'APLA') AND (({database}.roles.DELETE_FLAG)= 'N'))
    AND {database}.roles.mojap_snapshot_date = date '{snapshot_date}'
    AND {database}.parties.mojap_snapshot_date = date '{snapshot_date}'
    AND {database}.addresses.mojap_snapshot_date = date '{snapshot_date}';
"""

pydb.start_query_execution_and_wait(create_Adopt_Applicant_Info);



#### Applicant_Info validation

In [21]:
Adopt_Applicant_Info_count = pydb.read_sql_query("select count(*) as count from fcsq.Adopt_Applicant_Info")
Adopt_Applicant_Info_count

,count
0,253131


In [22]:
pydb.read_sql_query("select * from fcsq.adopt_applicant_info where case_number='HB12Z00228'")

,role,representative_role,role_model,party,case_number,person_given_first_name,person_family_name,company,postcode,gender,delete_flag
0,3232791,3437788,APLZ,2736019,HB12Z00228,Donald Garfield,Finnikin And B Pierre,<NA>,CR2 9BA,1,N
1,3232792,3437788,APLZ,2735994,HB12Z00228,Beverley Paula,Pierre,<NA>,CR2 9BA,2,N


## 10. adopt_respondent_info table - joins the roles, parties and address tables for respondents to get information on the respondents
<a name="adopt_respondent_info"></a>

### Drop the adopt_respondent_info table if it already exists and remove its data from the S3 bucket

In [23]:
drop_adopt_respondent_info = f"""
DROP TABLE IF EXISTS fcsq.adopt_respondent_info;
"""
pydb.start_query_execution_and_wait(drop_adopt_respondent_info)

# clean up previous adopt_respondent_info files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/adopt_respondent_info/").delete();

### Create the adopt_respondent_info table in Athena

In [24]:
create_adopt_respondent_info = f"""
CREATE TABLE IF NOT EXISTS fcsq.adopt_respondent_info
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/adopt_respondent_info') AS
SELECT DISTINCT
  {database}.roles.ROLE, 
  {database}.roles.REPRESENTATIVE_ROLE, 
  {database}.roles.ROLE_MODEL, 
  {database}.roles.PARTY, 
  {database}.roles.CASE_NUMBER, 
  {database}.parties.GENDER, 
  {database}.addresses.POSTCODE, 
  {database}.roles.DELETE_FLAG
FROM 
  ({database}.roles INNER JOIN {database}.parties ON {database}.roles.PARTY = {database}.parties.PARTY) 
    LEFT JOIN {database}.addresses ON {database}.roles.ADDRESS = {database}.addresses.ADDRESS
WHERE 
    ((({database}.roles.ROLE_MODEL)='RSPA') AND (({database}.roles.DELETE_FLAG)='N')) 
    OR ((({database}.roles.ROLE_MODEL)='RSPZ') AND (({database}.roles.DELETE_FLAG)='N'))
    AND {database}.roles.mojap_snapshot_date = date '{snapshot_date}'
    AND {database}.parties.mojap_snapshot_date = date '{snapshot_date}'
    AND {database}.addresses.mojap_snapshot_date = date '{snapshot_date}';
"""

pydb.start_query_execution_and_wait(create_adopt_respondent_info);



#### adopt_respondent_info validation

In [25]:
adopt_respondent_info_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_respondent_info")
adopt_respondent_info_count

,count
0,286085


In [26]:
pydb.read_sql_query("select * from fcsq.adopt_respondent_info where case_number='HB12Z00228'")

,role,representative_role,role_model,party,case_number,gender,postcode,delete_flag
0,3232794,<NA>,RSPZ,2736020,HB12Z00228,2,CR7 8JF,N
1,3232774,3606633,RSPZ,2736056,HB12Z00228,1,CR7 8JF,N


## 11. applicants 3 table - takes the adopt_applicant_info table and reformats it, decoding gender and representative role values into strings
<a name="applicants_3"></a>

In [27]:
create_adopt_applicants_1 = f"""
SELECT 	T1.role,
    T1.representative_role,
    T1.role_model,
    T1.party,
    T1.case_number,
    T1.gender,
    case when cast(gender as varchar(1)) = '1' then 'Male'
    when cast(gender as varchar(1)) = '2' then 'Female'
    else 'Unknown' end as Gender_Decode

    FROM fcsq.adopt_applicant_info AS t1
    ORDER BY t1.Case_Number;
"""
#pydb.start_query_execution_and_wait(create_adopt_applicants_1)
pydb.create_temp_table(create_adopt_applicants_1,'adopt_applicants_1')



create_adopt_applicants_2 = f"""
SELECT DISTINCT 
    T1.case_number,
    T1.party,
    max(T1.representative_role) as Rep_Role,
    max(T1.gender_decode) as Gender_Max
    from __temp__.adopt_applicants_1 AS t1
    group by Case_number, party;
"""

pydb.create_temp_table(create_adopt_applicants_2,'adopt_applicants_2')
#pydb.start_query_execution_and_wait(create_adopt_applicants_2)


create_adopt_applicants_3= f"""
SELECT t1.case_number,
    t1.party as App_Party_ID,
    t1.Rep_Role,
    t1.Gender_Max,
    case when t1.Rep_Role IS NULL then 'N'
    when t1.Rep_Role IS NOT NULL then 'Y'
    End as REPRESENTATION,
    case when Rep_Role IS NULL AND Gender_Max = 'Female' then 'Unrep_Female'
    when Rep_Role IS NULL AND Gender_Max = 'Male' then 'Unrep_Male'
    when Rep_Role IS NULL AND Gender_Max = 'Unknown' then 'Unrep_Unknown'
    when Rep_Role IS NOT NULL AND Gender_Max = 'Female' then 'Rep_Female'
    when Rep_Role IS NOT NULL AND Gender_Max = 'Male' then 'Rep_Male'
    when Rep_Role IS NOT NULL AND Gender_Max = 'Unknown' then 'Rep_Unknown'
    else '' end as App_Rep_Cat
    
    from __temp__.adopt_applicants_2 AS t1;


"""
#pydb.start_query_execution_and_wait(create_adopt_applicants_3)
pydb.create_temp_table(create_adopt_applicants_3,'adopt_applicants_3')


In [28]:
pydb.read_sql_query("SELECT count(*) as count from __temp__.adopt_applicants_3 where case_number='HB12Z00228'")

,count
0,2


## 12. Adopt_respondents temporary tables - takes the adopt_respondent_info table and reformats it, decoding gender and representative role values into strings
<a name="Adopt_respondents_temp"></a>

In [29]:
create_adopt_respondents_1 = f"""
SELECT T1.role,
    T1.representative_role,
    T1.role_model,
    T1.party,
    T1.case_number,
    T1.gender,
    case when cast(gender as varchar(1)) = '1' then 'Male'
    when cast(gender as varchar(1)) = '2' then 'Female'
    else 'Unknown' end as Gender_Decode

    FROM fcsq.adopt_respondent_info AS t1
    ORDER BY t1.Case_Number;
"""
#pydb.start_query_execution_and_wait(create_adopt_respondents_1)
pydb.create_temp_table(create_adopt_respondents_1,'adopt_respondents_1')



create_adopt_respondents_2 = f"""
    SELECT DISTINCT T1.case_number,
        T1.party,
        max(T1.representative_role) as Rep_Role,
        max(T1.gender_decode) as Gender_Max
    from __temp__.adopt_respondents_1 AS t1
    group by Case_number, party;
"""

pydb.create_temp_table(create_adopt_respondents_2,'adopt_respondents_2')
#pydb.start_query_execution_and_wait(create_adopt_respondents_2)


create_adopt_respondents_3= f"""
SELECT t1.case_number,
    t1.party as Resp_Party_ID,
    t1.Rep_Role,
    t1.Gender_Max,
    case when t1.Rep_Role IS NULL then 'N'
    when t1.Rep_Role IS NOT NULL then 'Y'
    End as REPRESENTATION,
    case when Rep_Role IS NULL AND Gender_Max = 'Female' then 'Unrep_Female'
    when Rep_Role IS NULL AND Gender_Max = 'Male' then 'Unrep_Male'
    when Rep_Role IS NULL AND Gender_Max = 'Unknown' then 'Unrep_Unknown'
    when Rep_Role IS NOT NULL AND Gender_Max = 'Female' then 'Rep_Female'
    when Rep_Role IS NOT NULL AND Gender_Max = 'Male' then 'Rep_Male'
    when Rep_Role IS NOT NULL AND Gender_Max = 'Unknown' then 'Rep_Unknown'
    else '' end as Resp_Rep_Cat
    
    from __temp__.adopt_respondents_2 AS t1;


"""
#pydb.start_query_execution_and_wait(create_adopt_respondents_3)
pydb.create_temp_table(create_adopt_respondents_3,'adopt_respondents_3')

## 13. Create adopt_app_rep table - joins the case data with the applicant representation data 
<a name="adopt_app_rep"></a>

In [30]:
adopt_app_rep_final = f"""
SELECT t1.YEAR, 
    t1.QUARTER,
    t1.CASE_NUMBER, 
    t1.Court,
    t2.App_Party_ID,
    t2.Representation,
    t2.Gender_Max as App_Gender,
    t2.App_Rep_Cat          
FROM __temp__.ADOPT_CASE_DATA_v3 t1
    LEFT JOIN __temp__.ADOPT_APPLICANTS_3 t2 ON (t1.CASE_NUMBER = t2.CASE_NUMBER);

"""

pydb.create_temp_table(adopt_app_rep_final,'adopt_app_rep_final')


In [31]:
pydb.read_sql_query("SELECT * FROM adopt_app_rep_final where case_number='HB12Z00228'")
#pydb.read_sql_query("SELECT * FROM adopt_case_data_v3 where case_number='HB12Z00228'")

,year,quarter,case_number,court,app_party_id,representation,app_gender,app_rep_cat
0,2012,4,HB12Z00228,150,2736019,Y,Male,Rep_Male
1,2012,4,HB12Z00228,150,2735994,Y,Female,Rep_Female


In [32]:
adopt_app_rep_final_check = "SELECT COUNT(*) as Count from __temp__.adopt_app_rep_final"
pydb.read_sql_query(adopt_app_rep_final_check)

,count
0,101841


## 14. Create adopt_resp_rep table - joins the case data with the respondent representation data
<a name="Adopt_resp_rep"></a>

In [33]:
adopt_resp_rep_final = f"""
   SELECT t1.YEAR, 
        t1.QUARTER,
        t1.CASE_NUMBER, 
        t1.Court,
          t2.Resp_Party_ID,
          t2.Representation,
          t2.Gender_Max as Resp_Gender,
          t2.Resp_Rep_Cat
          
      FROM __temp__.ADOPT_CASE_DATA_v3 t1
           LEFT JOIN __temp__.ADOPT_RESPONDENTS_3 t2 ON (t1.CASE_NUMBER = t2.CASE_NUMBER);
"""

pydb.create_temp_table(adopt_resp_rep_final,'adopt_resp_rep_final')

In [34]:
adopt_resp_rep_final_check = "SELECT COUNT(*) as Count from __temp__.adopt_resp_rep_final"
pydb.read_sql_query(adopt_resp_rep_final_check)

,count
0,101126


In [35]:
#pydb.read_sql_query("SELECT * FROM adopt_resp_rep_final where case_number='HB12Z00228'")
pydb.read_sql_query("SELECT * FROM adopt_case_data_v3 where case_number='HB12Z00228'")

,year,quarter,court,case_number,app_type,case_app_type,adoption,high_court,contested,number_applicants,adopter_type,app_date2,case_number_id
0,2012,4,150,HB12Z00228,AO,AO,Adoption,N,N,2,mixed-sex couple,06-12-2012,1


## 15. Adopt_Hearing_Events table - takes the adoption hearing events from the hearings table and joins it with the events table data 
<a name="Adopt_Hearing_Events"></a>

### Drop the Adopt_Hearing_Events table if it already exists and remove its data from the S3 bucket

In [36]:
drop_Adopt_Hearing_Events = f"""
DROP TABLE IF EXISTS fcsq.Adopt_Hearing_Events;
"""
pydb.start_query_execution_and_wait(drop_Adopt_Hearing_Events)

# clean up previous Adopt_Hearing_Events files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/Adopt_Hearing_Events/").delete();

### Create the Adopt_Hearing_Events table in Athena

In [37]:
create_Adopt_Hearing_Events = f"""
CREATE TABLE IF NOT EXISTS fcsq.Adopt_Hearing_Events
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Adopt_Hearing_Events') AS
SELECT {database}.hearings.EVENT,
  {database}.hearings.VACATED_FLAG,
  {database}.hearings.HEARING_TYPE,
  {database}.hearings.HEARING_DATE,
  {database}.events.RECEIPT_DATE,
  {database}.events.ERROR,
  {database}.events.CASE_NUMBER,
  {database}.events.EVENT_MODEL
FROM {database}.hearings
INNER JOIN {database}.events
ON {database}.hearings.EVENT            = {database}.events.EVENT
WHERE {database}.hearings.VACATED_FLAG IS NULL
AND {database}.events.ERROR             = 'N'
AND HEARING_DATE > date_parse('31-12-2009 00:00:00', '%d-%m-%Y %H:%i:%s')
AND (substring(case_number,5,1)='A' OR substring(case_number,5,1)='Z')
AND {database}.hearings.mojap_snapshot_date = date '{snapshot_date}' and {database}.events.mojap_snapshot_date = date '{snapshot_date}';
"""

pydb.start_query_execution_and_wait(create_Adopt_Hearing_Events);



#### Adopt_Hearing_Events validation

In [38]:
Adopt_Hearing_Events_count = pydb.read_sql_query("select count(*) as count from fcsq.Adopt_Hearing_Events")
Adopt_Hearing_Events_count

,count
0,510058


## 16. Adopt_Hearings_Cases table - takes the hearing events, filters by adoption event codes and adds a flag whether the hearing is the first in the case
<a name="Adopt_Hearings_Cases"></a>

### Drop the Adopt_Hearings_Cases table if it already exists and remove its data from the S3 bucket

In [39]:
drop_Adopt_Hearings_Cases = f"""
DROP TABLE IF EXISTS fcsq.Adopt_Hearings_Cases;
"""
pydb.start_query_execution_and_wait(drop_Adopt_Hearings_Cases)

# clean up previous Adopt_Hearings_Cases files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/Adopt_Hearings_Cases/").delete();

### Create the Adopt_Hearings_Cases table in Athena

In [40]:
"""
Equivalent to Hearings_Adopt_V3
"""

create_Adopt_Hearings_Cases = f"""
CREATE TABLE IF NOT EXISTS fcsq.Adopt_Hearings_Cases
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Adopt_Hearings_Cases') AS
select t1.case_number,
    t1.error,
    t1.event,
    t1.event_model,
    t1.hearing_date,
    t1.hearing_type,
    t1.receipt_date,
    t1.vacated_flag,
    substring(Case_Number,5,1) AS Case_Type
    from fcsq.Adopt_Hearing_Events AS t1
    where t1.event_model in ('A8', 'A90', 'A91', 'G60')
    order by t1.case_number, t1.receipt_date;
"""

pydb.start_query_execution_and_wait(create_Adopt_Hearings_Cases)

create_Adopt_Hearings_Cases_v2 = f"""
SELECT *,
(case when row_number() over (partition by Case_Number order by receipt_date) = 1 then 1 else 0 end) as Case_Number_ID
FROM fcsq.Adopt_Hearings_Cases
"""

pydb.create_temp_table(create_Adopt_Hearings_Cases_v2,'adopt_hearings_cases_v2')



#### Adopt_Hearings_Cases validation

In [41]:
Adopt_Hearings_Cases_count = pydb.read_sql_query("select count(*) as count from __temp__.Adopt_Hearings_Cases_v2")
Adopt_Hearings_Cases_count

,count
0,323515


## 17. Hearing_Adopt_Applicants table - joins the cases from applicant representation table to the first hearing for the case in the adopt hearings cases table 
<a name="Hearing_Adopt_Applicants"></a>

### Drop the Hearing_Adopt_Applicants table if it already exists and remove its data from the S3 bucket

In [42]:
drop_Hearing_Adopt_Applicants = f"""
DROP TABLE IF EXISTS fcsq.Hearing_Adopt_Applicants;
"""
pydb.start_query_execution_and_wait(drop_Hearing_Adopt_Applicants)

# clean up previous Hearing_Adopt_Applicants files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/Hearing_Adopt_Applicants/").delete();

### Create the Hearing_Adopt_Applicants table in Athena

In [43]:
create_Hearing_Adopt_Applicants = f"""
CREATE TABLE IF NOT EXISTS fcsq.Hearing_Adopt_Applicants
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Hearing_Adopt_Applicants') AS
SELECT t1.*,
t2.Case_Number_ID AS Hearing_Count
FROM __temp__.ADOPT_APP_REP_FINAL t1
LEFT JOIN __temp__.Adopt_Hearings_Cases_v2 t2 ON (t1.CASE_NUMBER = t2.CASE_NUMBER)
where t2.Case_Number_ID > 0;
"""

pydb.start_query_execution_and_wait(create_Hearing_Adopt_Applicants);



#### Hearing_Adopt_Applicants validation

In [44]:
Hearing_Adopt_Applicants_count = pydb.read_sql_query("select count(*) as count from __temp__.adopt_app_rep_final")
Hearing_Adopt_Applicants_count

,count
0,101841


## 18. Hearing_Adopt_Respondents table - joins the cases from respondents representation table to the first hearing for the case in the adopt hearings cases table 
<a name="Hearing_Adopt_Respondents"></a>

### Drop the Hearing_Adopt_Respondents table if it already exists and remove its data from the S3 bucket

In [45]:
drop_Hearing_Adopt_Respondents = f"""
DROP TABLE IF EXISTS fcsq.Hearing_Adopt_Respondents;
"""
pydb.start_query_execution_and_wait(drop_Hearing_Adopt_Respondents)

# clean up previous Hearing_Adopt_Respondents files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/Hearing_Adopt_Respondents/").delete();

### Create the Hearing_Adopt_Respondents table in Athena

In [46]:
create_Hearing_Adopt_Respondents = f"""
CREATE TABLE IF NOT EXISTS fcsq.Hearing_Adopt_Respondents
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Hearing_Adopt_Respondents') AS
    SELECT t1.*,
    t2.Case_Number_ID AS Hearing_Count
    FROM __temp__.ADOPT_RESP_REP_FINAL t1
    LEFT JOIN __temp__.Adopt_Hearings_Cases_v2 t2 ON (t1.CASE_NUMBER = t2.CASE_NUMBER)
    where t2.Case_Number_ID > 0;
"""

pydb.start_query_execution_and_wait(create_Hearing_Adopt_Respondents);



#### Hearing_Adopt_Respondents validation

In [47]:
Hearing_Adopt_Respondents_count = pydb.read_sql_query("select count(*) as count from fcsq.Hearing_Adopt_Respondents")
Hearing_Adopt_Respondents_count

,count
0,94976


## 19. Adopt_App table - Groups the hearing_adopt_applicants table and produces a count per group
<a name="Adopt_App"></a>

### Drop the Adopt_App table if it already exists and remove its data from the S3 bucket

In [48]:
drop_Adopt_App = f"""
DROP TABLE IF EXISTS fcsq.Adopt_App;
"""
pydb.start_query_execution_and_wait(drop_Adopt_App)

# clean up previous Adopt_App files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/Adopt_App/").delete();

### Create the Adopt_App table in Athena

In [49]:
create_Adopt_App = f"""
CREATE TABLE IF NOT EXISTS fcsq.Adopt_App
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Adopt_App') AS
SELECT
  'Adoption' AS CASE_TYPE,
  Year,
  Quarter,
  'Party' AS Category,
  'Applicant' AS PARTY,
   App_Gender AS Gender,
  Representation,
  Count (*) AS Count
FROM
  fcsq.HEARING_ADOPT_APPLICANTS
WHERE 
  Representation <> '' /*A very small number of cases from 2011/12 look into whether these should be recoded as N (gender is also blank)*/
GROUP BY
  'Adoption',
  Year,
  Quarter,
  'Party',
  'Applicant',
  App_Gender,
  Representation;
"""

pydb.start_query_execution_and_wait(create_Adopt_App);



#### Adopt_App validation

In [50]:
Adopt_App_count = pydb.read_sql_query("select count(*) as count from fcsq.Adopt_App")
Adopt_App_count

,count
0,221


## 20. Adopt_resp table - Groups the hearing_adopt_respondents table and produces a count per group
<a name="Adopt_resp"></a>

### Drop the Adopt_resp table if it already exists and remove its data from the S3 bucket

In [51]:
drop_Adopt_resp = f"""
DROP TABLE IF EXISTS fcsq.Adopt_resp;
"""
pydb.start_query_execution_and_wait(drop_Adopt_resp)

# clean up previous Adopt_resp files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/Adopt_resp/").delete();

### Create the Adopt_resp table in Athena

In [52]:
create_Adopt_resp = f"""
CREATE TABLE IF NOT EXISTS fcsq.Adopt_resp
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Adopt_resp') AS
SELECT
  'Adoption' AS CASE_TYPE,
  Year,
  Quarter,
  'Party' AS Category,
  'Respondent' AS PARTY,
  Resp_Gender AS Gender,
  Representation,
  Count (*) AS Count
FROM
  fcsq.HEARING_ADOPT_RESPONDENTS
WHERE 
  Representation <> '' /*A very small number of cases from 2011/12 look into whether these should be recoded as N (gender is also blank)*/
GROUP BY
  'Adoption',
  Year,
  Quarter,
  'Party',
  'Applicant',
  Resp_Gender,
  Representation;
"""

pydb.start_query_execution_and_wait(create_Adopt_resp);



#### Adopt_resp validation

In [53]:
Adopt_resp_count = pydb.read_sql_query("select count(*) as count from fcsq.Adopt_resp")
Adopt_resp_count

,count
0,233


## 21. adopt_case table - groups and formats adopt_case_data_v3 table and gives a count for each group
<a name="adopt_case"></a>

### Drop the adopt_case table if it already exists and remove its data from the S3 bucket

In [54]:
drop_adopt_case = f"""
DROP TABLE IF EXISTS fcsq.adopt_case;
"""
pydb.start_query_execution_and_wait(drop_adopt_case)

# clean up previous adopt_case files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/Adopt_case/").delete();

### Create the adopt_case table in Athena

In [55]:
create_adopt_case = f"""
CREATE TABLE IF NOT EXISTS fcsq.Adopt_case
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Adopt_case') AS
SELECT *,
    Count(*) as Count FROM
    (SELECT
        'Adoption' AS CASE_TYPE,
        Year,
        Quarter,
        'Cases' AS Category,
        'N/A' AS PARTY,
        'N/A' AS Gender,
        'N/A' AS Representation
    FROM
      __temp__.adopt_case_data_v3)
GROUP BY
  CASE_TYPE,
  Year,
  Quarter,
  Category,
  PARTY,
  Gender,
  Representation;
"""

pydb.start_query_execution_and_wait(create_adopt_case);



#### adopt_case validation

In [56]:
adopt_case_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_case")
adopt_case_count

,count
0,43


## 22. Adopt_Case_Hearings table - creates a count of all the cases with a hearing per quarter
<a name="Adopt_Case_Hearings"></a>

### Drop the Adopt_Case_Hearings table if it already exists and remove its data from the S3 bucket

In [57]:
drop_Adopt_Case_Hearings = f"""
DROP TABLE IF EXISTS fcsq.Adopt_Case_Hearings;
"""
pydb.start_query_execution_and_wait(drop_Adopt_Case_Hearings)

# clean up previous Adopt_Case_Hearings files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/Adopt_Case_Hearings/").delete();

### Create the Adopt_Case_Hearings table in Athena

In [58]:
create_hearing_adopt_case =f"""
SELECT DISTINCT Year, Quarter, Case_Number
FROM fcsq.HEARING_ADOPT_Applicants;
"""

pydb.create_temp_table(create_hearing_adopt_case,'hearing_adopt_case')



create_Adopt_Case_Hearings = f"""
CREATE TABLE IF NOT EXISTS fcsq.Adopt_Case_Hearings
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Adopt_Case_Hearings') AS
SELECT *, Count(*) as Count FROM
    (SELECT
      'Adoption' AS CASE_TYPE,
      Year,
      Quarter,
      'Cases with a hearing' AS Category,
      'N/A' AS PARTY,
      'N/A' AS Gender,
      'N/A' AS Representation
    FROM
      __temp__.Hearing_ADOPT_Case)
GROUP BY
  CASE_TYPE,
  Year,
  Quarter,
  Category,
  PARTY,
  Gender,
  Representation;
"""

pydb.start_query_execution_and_wait(create_Adopt_Case_Hearings);



#### Adopt_Case_Hearings validation

In [59]:
Adopt_Case_Hearings_count = pydb.read_sql_query("select count(*) as count from fcsq.Adopt_Case_Hearings")
Adopt_Case_Hearings_count

,count
0,43


## 23. Adoption table - Joins the applicant/respondent representation count tables, case count table, and case hearing count tables 
<a name="Adoption"></a>

### Drop the Adoption table if it already exists and remove its data from the S3 bucket

In [60]:
drop_Adoption = f"""
DROP TABLE IF EXISTS fcsq.Adoption;
"""
pydb.start_query_execution_and_wait(drop_Adoption)

# clean up previous Adoption files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/Adoption/").delete();

### Create the Adoption table in Athena

In [61]:
create_Adoption = f"""
CREATE TABLE IF NOT EXISTS fcsq.Adoption
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Adoption') AS
SELECT
  *
FROM
 fcsq.ADOPT_APP
UNION ALL
SELECT
  *
FROM
  fcsq.ADOPT_RESP
UNION ALL
SELECT
  *
FROM
  fcsq.ADOPT_CASE
UNION ALL
SELECT
  *
FROM
  fcsq.ADOPT_CASE_HEARINGS;
"""

pydb.start_query_execution_and_wait(create_Adoption);



#### Adoption validation

In [62]:
Adoption_count = pydb.read_sql_query("select count(*) as count from fcsq.Adoption")
Adoption_count

,count
0,540


## 24. Applicant_representation table - creates a table showing whether all, some, or none of the applicants for a case have representation
<a name="Applicant_representation"></a>

### Drop the Applicant_representation table if it already exists and remove its data from the S3 bucket

In [63]:
drop_Applicant_representation = f"""
DROP TABLE IF EXISTS fcsq.Applicant_representation;
"""
pydb.start_query_execution_and_wait(drop_Applicant_representation)

# clean up previous Applicant_representation files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/Applicant_representation/").delete();

### Create the Applicant_representation table in Athena

In [64]:
create_applicants_1 = f"""
SELECT Distinct t1.Case_Number, t1.Party, MAX(t1.Representative_Role) as Max_Rep_Role
FROM fcsq.Adopt_Applicant_Info t1
Group by Case_Number, Party;
"""

pydb.create_temp_table(create_applicants_1,'applicants_1')

create_applicants_2 = f"""
SELECT  t1.*,
case when Max_Rep_Role IS NULL then 0
else 1
end as Rep_IND
FROM __temp__.Applicants_1 t1;
"""
pydb.create_temp_table(create_applicants_2,'applicants_2')

create_applicants_3 = f"""
SELECT Distinct t1.Case_Number,
Count(t1.Party) as CountOfParty,
SUM(t1.Rep_Ind) as SumOfRep_IND
FROM __temp__.Applicants_2 t1
Group by Case_Number;
"""
pydb.create_temp_table(create_applicants_3,'applicants_3')

create_Applicant_representation = f"""
CREATE TABLE IF NOT EXISTS fcsq.Applicant_representation
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Applicant_representation') AS
SELECT t1.Case_Number,
t1.CountOfParty,
t1.SumOfRep_IND,
CASE WHEN t1.SumOfRep_Ind > t1.CountOfParty then 'Error'
WHEN t1.SumOfRep_Ind = t1.CountOfParty then 'All'
WHEN t1.SumOfRep_Ind =0 then 'None' else 'Some'  end as App_Rep_Cat
FROM __temp__.Applicants_3 t1;
"""

pydb.start_query_execution_and_wait(create_Applicant_representation);



#### Applicant_representation validation

In [65]:
Applicant_representation_count = pydb.read_sql_query("select count(*) as count from fcsq.Applicant_representation")
Applicant_representation_count

,count
0,180608


## 25. Respondent_Representation table - creates a table showing whether all, some, or none of the respondents for a case have representation
<a name="Respondent_representation"></a>

### Drop the Respondent_Representation table if it already exists and remove its data from the S3 bucket

In [66]:
drop_Respondent_Representation = f"""
DROP TABLE IF EXISTS fcsq.Respondent_Representation;
"""
pydb.start_query_execution_and_wait(drop_Respondent_Representation)

# clean up previous Respondent_Representation files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/Respondent_Representation/").delete();

### Create the Respondent_Representation table in Athena

In [67]:
create_respondents_1 = f"""
SELECT Distinct t1.Case_Number, t1.Party, MAX(t1.Representative_Role) as Max_Rep_Role
FROM fcsq.Adopt_Respondent_Info t1
Group by Case_Number, Party;
"""

pydb.create_temp_table(create_respondents_1,'respondents_1')

create_respondents_2 = f"""
SELECT  t1.*,
case when Max_Rep_Role IS NULL then 0
else 1
end as Rep_IND
FROM __temp__.respondents_1 t1;
"""
pydb.create_temp_table(create_respondents_2,'respondents_2')

create_respondents_3 = f"""
SELECT Distinct t1.Case_Number,
Count(t1.Party) as CountOfParty,
SUM(t1.Rep_Ind) as SumOfRep_IND
FROM __temp__.respondents_2 t1
Group by Case_Number;
"""
pydb.create_temp_table(create_respondents_3,'respondents_3')

create_Respondent_Representation = f"""
CREATE TABLE IF NOT EXISTS fcsq.Respondent_Representation
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Respondent_Representation') AS
SELECT t1.Case_Number,
t1.CountOfParty,
t1.SumOfRep_IND,
CASE WHEN t1.SumOfRep_Ind > t1.CountOfParty then 'Error'
WHEN t1.SumOfRep_Ind = t1.CountOfParty then 'All'
WHEN t1.SumOfRep_Ind =0 then 'None' else 'Some'  end as Res_Rep_Cat
FROM __temp__.Respondents_3 t1
"""

pydb.start_query_execution_and_wait(create_Respondent_Representation);



#### Respondent_Representation validation

In [68]:
Respondent_Representation_count = pydb.read_sql_query("select count(*) as count from fcsq.Respondent_Representation")
Respondent_Representation_count

,count
0,164479


## 26. Adopt_Disposals_Final table - joins the adopt_apps_and_orders_match table and the representation tables, joining timeliness and legal representation data for a case together
<a name="Adopt_Disposals_Final"></a>

In [69]:
create_ADOPT_APP_AND_ORDERS_WITH_REP = f"""
SELECT t1.*,
t2.APP_REP_CAT, 
t3.RES_REP_CAT
FROM fcsq.ADOPT_APPS_AND_ORDERS_MATCH AS t1
LEFT JOIN fcsq.Applicant_Representation AS t2 ON (t1.CASE_NUMBER = t2.CASE_NUMBER)
LEFT JOIN fcsq.RESPONDENT_REPRESENTATION as t3 ON t1.CASE_NUMBER = t3.CASE_NUMBER;
"""
pydb.create_temp_table(create_ADOPT_APP_AND_ORDERS_WITH_REP,'ADOPT_APP_AND_ORDERS_WITH_REP')

In [113]:
pydb.read_sql_query("SELECT * from __temp__.adopt_app_and_orders_with_rep WHERE case_number in ('MB20Z01235','MB20Z01236')")

,case_number,app_type,court,app_date,disp_date,wait_weeks,disp_type,year,quarter,order_type,adoption,type,dsp_court,app_rep_cat,res_rep_cat
0,MB20Z01235,AO,270,2020-07-25,2020-10-06,10.428571,A73,2020,4,OFC,Non-adoption,,270,None,None
1,MB20Z01236,AO,270,2020-07-25,2020-11-02,14.285714,A76,2020,4,<NA>,Adoption,Standard,270,None,None


In [71]:
pydb.read_sql_query("SELECT Count(*) as COUNT from __temp__.adopt_app_and_orders_with_rep")

,count
0,147875


In [72]:
create_Adopt_Disposals_Final = f"""
SELECT t1.*,
t2.Region_Pre2014, 
t2.Region,
cast(t1.Year as varchar(3)) || '-Q' || cast(t1.quarter as varchar(3)) AS Quarter2, 
case when (t1.APP_REP_CAT Is Null Or t1.RES_REP_CAT Is Null)  then '5 Unknown'
    when t1.APP_REP_CAT='None' and t1.RES_REP_CAT ='None' then '4 Neither'
    when t1.APP_REP_CAT='None' and t1.RES_REP_CAT != 'None' then '3 Respondent Only'
    when t1.APP_REP_CAT != 'None' and t1.RES_REP_CAT = 'None' then '2 Applicant Only'
Else '1 Both'
End AS REP_CAT,
Case when t1.YEAR < 2014 then t2.Region_Pre2014
Else t2.Region
End As Final_Region

FROM __temp__.ADOPT_APP_AND_ORDERS_WITH_REP AS t1 LEFT JOIN fcsq.COURT_MV_FEB21_DFJ as t2
ON t1.DSP_COURT = cast(t2.Code as varchar(3));

"""

pydb.create_temp_table(create_Adopt_Disposals_Final,'Adopt_Disposals_Final')


create_Adopt_Disposals_Final_2 = f"""
SELECT *
FROM __temp__.ADOPT_DISPOSALS_FINAL
WHERE adoption = 'Adoption';
"""

pydb.create_temp_table(create_Adopt_Disposals_Final_2,'Adopt_Disposals_Final_2')



In [73]:
pydb.read_sql_query(f"SELECT * FROM {database}.courts_mv WHERE {database}.courts_mv.mojap_snapshot_date = date '{snapshot_date}' AND code=150")

,court,code,name,tel_no,fax_no,live,dx_number,sups_centralised_flag,default_printer,district_registry,fap_id,deed_pack_number,welsh_high_court_name,welsh_county_court_name,lov_name,fpc_court_indicator,a_rowid,b_rowid,welsh_court_name,dr_tel_no,open_from,closed_at,dr_open_from,dr_closed_at,by_appointment_ind,dfj_area_id,mojap_file_land_timestamp,mojap_snapshot_date
0,BN,150,BRIGHTON,01273-674421,0870 324 0324,Y,98070 BRIGHTON 3,Y,CA325758BP-PRN0724,Y,PRN0724,7,<NA>,<NA>,BRIGHTON,N,AAAPFLAAJAAACgnAAD,AAAPFLAAJAAACgnAAD,<NA>,01273-674421,<NA>,<NA>,<NA>,<NA>,Y,BRG,1629395297,2021-08-19
1,HB,150,BRIGHTON,01273 811333,01273 607638,N,142600 BRIGHTON-12,Y,CA325758BP-PRN0724,Y,PRN0724,7,<NA>,<NA>,BRIGHTON,N,AAAPFLAAJAAACgnAAD,AAAPFLAAJAAACgnAAD,<NA>,01273 811333,<NA>,<NA>,<NA>,<NA>,Y,BRG,1629395297,2021-08-19


In [74]:
pydb.read_sql_query("SELECT * FROM __temp__.Adopt_Disposals_Final where CASE_NUMBER='HB12Z00228'")

,case_number,app_type,court,app_date,disp_date,wait_weeks,disp_type,year,quarter,order_type,adoption,type,dsp_court,app_rep_cat,res_rep_cat,region_pre2014,region,quarter2,rep_cat,final_region
0,HB12Z00228,AO,150,2012-12-06,2013-03-27,15.857143,A76,2013,1,<NA>,Adoption,Standard,150,All,Some,SOUTH EAST,SOUTH EAST,2013-Q1,1 Both,SOUTH EAST


In [75]:
pydb.read_sql_query("SELECT * FROM __temp__.Adopt_Disposals_Final_2 where Case_number='SQ17Z00141'")

,case_number,app_type,court,app_date,disp_date,wait_weeks,disp_type,year,quarter,order_type,adoption,type,dsp_court,app_rep_cat,res_rep_cat,region_pre2014,region,quarter2,rep_cat,final_region
0,SQ17Z00141,AO,338,2017-08-29,2018-01-08,18.857143,A76,2018,1,<NA>,Adoption,Standard,338,All,Some,MIDLANDS,MIDLANDS,2018-Q1,1 Both,MIDLANDS


In [76]:
pydb.read_sql_query("SELECT * FROM __temp__.Adopt_Disposals_Final_2 ORDER BY wait_weeks desc")

,case_number,app_type,court,app_date,disp_date,wait_weeks,disp_type,year,quarter,order_type,adoption,type,dsp_court,app_rep_cat,res_rep_cat,region_pre2014,region,quarter2,rep_cat,final_region
0,BV00A00280,SP,249,2000-04-11,2001-01-15,39.857143,A15,2001,1,<NA>,Adoption,Standard,249,None,None,MIDLANDS,MIDLANDS,2001-Q1,4 Neither,MIDLANDS
1,CM13Z00421,AO,167,2013-12-05,2014-09-10,39.857143,A76,2014,3,<NA>,Adoption,Standard,167,All,None,SOUTH EAST,SOUTH EAST,2014-Q3,2 Applicant Only,SOUTH EAST
2,CM13Z00422,AO,167,2013-12-05,2014-09-10,39.857143,A76,2014,3,<NA>,Adoption,Standard,167,None,None,SOUTH EAST,SOUTH EAST,2014-Q3,4 Neither,SOUTH EAST
3,LN14Z00583,AO,249,2014-08-14,2015-05-20,39.857143,A76,2015,2,<NA>,Adoption,Standard,249,None,Some,MIDLANDS,MIDLANDS,2015-Q2,3 Respondent Only,MIDLANDS
4,PO14Z00308,AO,302,2014-12-09,2015-09-14,39.857143,A76,2015,3,<NA>,Adoption,Standard,302,All,None,SOUTH WEST,SOUTH WEST,2015-Q3,2 Applicant Only,SOUTH WEST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80644,LS16Z00014,AO,243,2016-01-15,2016-04-05,11.571429,A76,2016,2,<NA>,Adoption,Standard,243,None,None,NORTH EAST,NORTH EAST,2016-Q2,4 Neither,NORTH EAST
80645,PE02A01163,SP,294,2002-09-16,2002-12-06,11.571429,A15,2002,4,<NA>,Adoption,Standard,294,None,<NA>,SOUTH EAST,SOUTH EAST,2002-Q4,5 Unknown,SOUTH EAST
80646,PO18Z00020,AO,302,2018-01-15,2018-04-06,11.571429,A76,2018,2,<NA>,Adoption,Standard,302,None,None,SOUTH WEST,SOUTH WEST,2018-Q2,4 Neither,SOUTH WEST
80647,MA16Z00232,AO,262,2016-04-18,2016-07-08,11.571429,A76,2016,3,<NA>,Adoption,Standard,262,None,None,NORTH WEST,NORTH WEST,2016-Q3,4 Neither,NORTH WEST


In [98]:
pydb.read_sql_query("SELECT distinct app_type FROM __temp__.Adopt_Disposals_Final_2")

,app_type
0,"AO, PT9"
1,PLA
2,"AO, CCS"
3,PF
4,"PT9, AO"
5,RPLA
6,CCS
7,"CA, AO"
8,AD
9,"OR, AO"


In [158]:
df =pydb.read_sql_query("select * from __temp__.Adopt_Disposals_Final_2 order by case_number")
df.to_csv(path_or_buf = '~/FCSQ_data/debug.csv',index=False)

In [157]:
pydb.read_sql_query("select * from __temp__.Adopt_Disposals_Final_2 where case_number='MB20Z01363'")

,case_number,app_type,court,app_date,disp_date,wait_weeks,disp_type,year,quarter,order_type,adoption,type,dsp_court,app_rep_cat,res_rep_cat,region_pre2014,region,quarter2,rep_cat,final_region
0,MB20Z01363,AO,270,2020-08-15,2020-10-06,7.428571,A76,2020,4,<NA>,Adoption,Standard,270,None,None,NORTH EAST,NORTH EAST,2020-Q4,4 Neither,NORTH EAST


#### Adopt_Disposals_Final validation

In [77]:
Adopt_Disposals_Final_count = pydb.read_sql_query("select count(*) as count from __temp__.Adopt_Disposals_Final_2")
Adopt_Disposals_Final_count

,count
0,80649


In [97]:
pydb.read_sql_query("select * from __temp__.Adopt_Disposals_Final_2 where rep_cat='4 Neither' and year=2020 and quarter=4 order by wait_weeks desc")
#check.to_csv(path_or_buf = '~/FCSQ_data/check.csv',index=False)

,case_number,app_type,court,app_date,disp_date,wait_weeks,disp_type,year,quarter,order_type,adoption,type,dsp_court,app_rep_cat,res_rep_cat,region_pre2014,region,quarter2,rep_cat,final_region
0,BO09A01358,PL,321,2009-12-03,2020-10-01,565.000000,A15,2020,4,<NA>,Adoption,Standard,321,None,None,LONDON,LONDON,2020-Q4,4 Neither,LONDON
1,WV18Z00352,AO,378,2018-10-26,2020-10-05,101.428571,A76,2020,4,<NA>,Adoption,Standard,378,None,None,MIDLANDS,MIDLANDS,2020-Q4,4 Neither,MIDLANDS
2,YO19Z00069,AO,386,2019-02-08,2020-10-28,89.714286,A76,2020,4,<NA>,Adoption,Standard,386,None,None,NORTH EAST,NORTH EAST,2020-Q4,4 Neither,NORTH EAST
3,YO19Z00068,AO,386,2019-02-08,2020-10-28,89.714286,A76,2020,4,<NA>,Adoption,Standard,386,None,None,NORTH EAST,NORTH EAST,2020-Q4,4 Neither,NORTH EAST
4,YO19Z00067,AO,386,2019-02-08,2020-10-28,89.714286,A76,2020,4,<NA>,Adoption,Standard,386,None,None,NORTH EAST,NORTH EAST,2020-Q4,4 Neither,NORTH EAST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,MB20Z01892,AO,270,2020-11-17,2020-12-16,4.142857,A76,2020,4,<NA>,Adoption,Standard,270,None,None,NORTH EAST,NORTH EAST,2020-Q4,4 Neither,NORTH EAST
993,MB20Z01893,AO,270,2020-11-17,2020-12-16,4.142857,A76,2020,4,<NA>,Adoption,Standard,270,None,None,NORTH EAST,NORTH EAST,2020-Q4,4 Neither,NORTH EAST
994,MB20Z01894,AO,270,2020-11-17,2020-12-16,4.142857,A76,2020,4,<NA>,Adoption,Standard,270,None,None,NORTH EAST,NORTH EAST,2020-Q4,4 Neither,NORTH EAST
995,SA20Z01706,AO,344,2020-11-24,2020-12-18,3.428571,A76,2020,4,<NA>,Adoption,Standard,344,None,None,WALES,WALES,2020-Q4,4 Neither,WALES


In [151]:
pydb.read_sql_query("SELECT * FROM adopt_disposals_final_2 where case_number='GL20Z00165'")

,case_number,app_type,court,app_date,disp_date,wait_weeks,disp_type,year,quarter,order_type,adoption,type,dsp_court,app_rep_cat,res_rep_cat,region_pre2014,region,quarter2,rep_cat,final_region
0,GL20Z00165,CA,203,2020-03-12,2021-03-04,51.0,A76,2021,1,<NA>,Adoption,?Foreign?,203,None,None,SOUTH WEST,SOUTH WEST,2021-Q1,4 Neither,SOUTH WEST


In [160]:
pydb.read_sql_query("select * from __temp__.Adopt_Disposals_Final_2 where case_number='MB20Z01595'")


,case_number,app_type,court,app_date,disp_date,wait_weeks,disp_type,year,quarter,order_type,adoption,type,dsp_court,app_rep_cat,res_rep_cat,region_pre2014,region,quarter2,rep_cat,final_region
0,MB20Z01595,AO,270,2020-09-23,2020-11-23,8.714286,A76,2020,4,<NA>,Adoption,?Foreign?,270,All,None,NORTH EAST,NORTH EAST,2020-Q4,2 Applicant Only,NORTH EAST


## 27. Adopt_Quarterly table - groups the disposals_final table by quarter, providing a total and mean wait_weeks per quarter
<a name="Adopt_Quarterly"></a>

### Drop the Adopt_Quarterly table if it already exists and remove its data from the S3 bucket

In [79]:
drop_Adopt_Quarterly = f"""
DROP TABLE IF EXISTS fcsq.Adopt_Quarterly;
"""
pydb.start_query_execution_and_wait(drop_Adopt_Quarterly)

# clean up previous Adopt_Quarterly files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/Adopt_Quarterly/").delete();

### Create the Adopt_Quarterly table in Athena

In [80]:
create_Adopt_Quarterly = f"""
CREATE TABLE IF NOT EXISTS fcsq.Adopt_Quarterly
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Adopt_Quarterly') AS
SELECT DISTINCT
        'Adoption' as type,
        year,
        'Q' || cast(quarter as varchar(3)) AS quarter,
        rep_cat,
        count(*) as n,
        avg(wait_weeks) as mean
    FROM 
        __temp__.ADOPT_DISPOSALS_FINAL_2
    WHERE year > 2010 
    AND quarter2 <> '2021-Q4'
    AND adoption='Adoption'
GROUP BY
    year,
    quarter,
    rep_cat
    
UNION ALL
SELECT DISTINCT
        'Adoption' as type,
        year,
        'Q' || cast(quarter as varchar(3)) AS quarter,
        'All' as rep_cat,
        count(*) as n,
        avg(wait_weeks) as mean
    FROM 
        __temp__.ADOPT_DISPOSALS_FINAL_2
    WHERE year > 2010 
    AND quarter2 <> '2021-Q4'
    AND adoption='Adoption'
GROUP BY
    year,
    quarter
"""


testA = f"""
SELECT * FROM __temp__.ADOPT_DISPOSALS_FINAL_2 WHERE year=2015 AND quarter = 3;
"""
testB = f"""
SELECT * FROM fcsq.Adopt_Quarterly WHERE year=2015 AND quarter = 'Q3' ORDER BY rep_cat
"""

pydb.start_query_execution_and_wait(create_Adopt_Quarterly)
#pydb.read_sql_query(testA)
pydb.read_sql_query(testB)





,type,year,quarter,rep_cat,n,mean
0,Adoption,2015,Q3,1 Both,16,28.651786
1,Adoption,2015,Q3,2 Applicant Only,59,21.380145
2,Adoption,2015,Q3,3 Respondent Only,44,28.032468
3,Adoption,2015,Q3,4 Neither,1326,14.965740
4,Adoption,2015,Q3,5 Unknown,15,20.438095
5,Adoption,2015,Q3,All,1460,15.824951


#### Adopt_Quarterly validation

In [81]:
Adopt_Quarterly_count = pydb.read_sql_query("select count(*) as count from fcsq.Adopt_Quarterly")
Adopt_Quarterly_count

,count
0,258


In [82]:
pydb.read_sql_query("select * from fcsq.Adopt_Quarterly WHERE year = 2014 AND quarter = \'Q2' ORDER BY rep_cat")

,type,year,quarter,rep_cat,n,mean
0,Adoption,2014,Q2,1 Both,17,35.344538
1,Adoption,2014,Q2,2 Applicant Only,56,24.778061
2,Adoption,2014,Q2,3 Respondent Only,44,23.863636
3,Adoption,2014,Q2,4 Neither,1539,14.656734
4,Adoption,2014,Q2,5 Unknown,15,12.047619
5,Adoption,2014,Q2,All,1671,15.425408


## 28. Adopt_Annual table - groups the disposals_final table annually, providing a total and mean wait_weeks annually
<a name="Adopt_Annual"></a>

### Drop the Adopt_Annual table if it already exists and remove its data from the S3 bucket

In [83]:
drop_Adopt_Annual = f"""
DROP TABLE IF EXISTS fcsq.Adopt_Annual;
"""
pydb.start_query_execution_and_wait(drop_Adopt_Annual)

# clean up previous Adopt_Annual files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/Adopt_Annual/").delete();

### Create the Adopt_Annual table in Athena

In [84]:
create_Adopt_Annual = f"""
CREATE TABLE IF NOT EXISTS fcsq.Adopt_Annual
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Adopt_Annual') AS
SELECT DISTINCT
        'Adoption' as type,
        year,
        'N/A' as quarter,
        rep_cat,
        count(*) as n,
        avg(wait_weeks) as mean
    FROM 
        __temp__.ADOPT_DISPOSALS_FINAL_2
    WHERE year > 2010 
    AND year < 2021
    AND adoption='Adoption'
GROUP BY
    year,
    rep_cat
    
UNION ALL
SELECT DISTINCT
        'Adoption' as type,
        year,
        'N/A' as quarter,
        'All' as rep_cat,
        count(*) as n,
        avg(wait_weeks) as mean
    FROM 
        __temp__.ADOPT_DISPOSALS_FINAL_2
    WHERE year > 2010 
    AND year < 2021
    AND adoption='Adoption'
GROUP BY
    year
"""

pydb.start_query_execution_and_wait(create_Adopt_Annual);



#### Adopt_Annual validation

In [85]:
Adopt_Annual_count = pydb.read_sql_query("select count(*) as count from fcsq.Adopt_Annual")
Adopt_Annual_count



,count
0,60


In [86]:
df = pydb.read_sql_query("SELECT * FROM fcsq.Adoption")
df.to_csv(path_or_buf = 's3://alpha-family-data/CSVs/Adoption_legrep.csv',index=False)
#df.to_excel('adoption.xlsx')


In [87]:
df = pydb.read_sql_query("SELECT * FROM fcsq.Adopt_Annual UNION ALL SELECT * FROM fcsq.Adopt_Quarterly ORDER BY type,year,quarter,rep_cat")
df.to_csv(path_or_buf = '~/FCSQ_data/timeliness.csv',index=False)

## 29. adopt_timeliness_combined table - combines and orders the annual and quarterly timeliness data 
<a name="Adopt_timeliness_combined"></a>

### Drop the adopt_timeliness_combined table if it already exists and remove its data from the S3 bucket

In [88]:
drop_adopt_timeliness_combined = f"""
DROP TABLE IF EXISTS fcsq.adopt_timeliness_combined;
"""
pydb.start_query_execution_and_wait(drop_adopt_timeliness_combined)

# clean up previous adopt_timeliness_combined files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/adopt_timeliness_combined/").delete();

### Create the adopt_timeliness_combined table in Athena

In [89]:
create_adopt_timeliness_combined = f"""
CREATE TABLE IF NOT EXISTS fcsq.adopt_timeliness_combined
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/adopt_timeliness_combined') AS
SELECT * FROM fcsq.Adopt_Annual 
UNION ALL 
SELECT * FROM fcsq.Adopt_Quarterly ORDER BY type,year,quarter,rep_cat
"""

pydb.start_query_execution_and_wait(create_adopt_timeliness_combined);



#### adopt_timeliness_combined validation

In [90]:
adopt_timeliness_combined_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_timeliness_combined")
adopt_timeliness_combined_count

,count
0,318


In [91]:
df = pydb.read_sql_query("SELECT * FROM fcsq.adopt_timeliness_combined WHERE (Year<2021 OR Quarter!='Q3') AND (Year<2021 OR Quarter!='Q2')")
df = df.pivot_table(index=['type','year','quarter'],columns=['rep_cat'],values = ['n','mean'],aggfunc=sum, fill_value=0).swaplevel(axis=1).sort_index(axis=1)
df

rep_cat               1 Both                2 Applicant Only       3 Respondent Only      4 Neither               5 Unknown              All                    
                                   mean   n              mean    n              mean    n              mean     n              mean    n              mean     n
type     year quarter                                                                                                                                           
Adoption 2011 N/A             28.905775  94         19.257370  252         19.077849  178         15.529558  3968         15.033223  172         16.117679  4664
              Q1              32.278195  19         21.970522   63         20.758242   39         15.853214   909         14.145963   46         16.606213  1076
              Q2              32.619048  18         19.384615   52         16.620130   44         14.730476  1050         15.293233   38         15.286665  1202
              Q3              26.396825  27         16.682540   63         20.229236   43         15.817739   968         14.298030   58         16.198817  1159
              Q4              26.800000  30         19.050193   74         18.945055   52         15.784960  1041         17.485714   30         16.426709  1227
         2012 N/A             29.933333  75         23.284467  229         19.105551  157         14.430015  4652         18.880272  105         15.271669  5218
              Q1              27.296703  13         22.721550   59         20.084249   39         14.799785  1061         17.520000   25         15.554959  1197
              Q2              30.123810  15         26.263039   63         17.937729   39         14.487096  1096         20.220408   35         15.538118  1248
              Q3              33.366460  23         21.293956   52         21.833977   37         14.630694  1198         19.529762   24         15.501392  1334
              Q4              27.952381  24         22.358442   55         16.877551   42         13.893931  1297         17.523810   21         14.591979  1439
         2013 N/A             28.203810  75         20.768116  207         19.145677  152         14.196296  5531         15.414747   62         14.733675  6027
              Q1              37.376623  22         21.320106   54         21.282738   48         14.628715  1192         19.129870   11         15.556142  1327
              Q2              23.292208  22         18.232143   56         18.094340   53         14.060805  1459         14.731429   25         14.473950  1615
              Q3              29.659341  13         19.103448   58         18.464286   28         13.576372  1445         17.704762   15         14.043618  1559
              Q4              21.944444  18         26.120879   39         17.937888   23         14.599104  1435         10.129870   11         14.998315  1526
         2014 N/A             30.596059  58         22.310924  238         23.214286  196         15.263089  6131         31.181406   63         16.030063  6686
              Q1              20.285714   5         18.060440   52         22.109023   38         14.592715  1510         14.983193   17         14.901621  1622
              Q2              35.344538  17         24.778061   56         23.863636   44         14.656734  1539         12.047619   15         15.425408  1671
              Q3              28.849206  18         21.009070   63         22.827068   57         16.589554  1600         82.704762   15         17.642816  1753
              Q4              30.722222  18         24.771855   67         23.837093   57         15.143725  1482         18.026786   16         16.038328  1640
         2015 N/A             29.309927  59         20.390756  204         24.996811  224         15.255076  5572         15.863265   70         15.924294  6129
              Q1              27.846154  13         21.018433   62         24.060268   64         15.052715  1439         12.957143   20

In [92]:
df.to_csv(path_or_buf = '~/FCSQ_data/timeliness.csv',index=False)